In [1]:
import pandas as pd

In [2]:
triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'

In [3]:
from sklearn import datasets, svm, tree, preprocessing, metrics
import sklearn.ensemble as ske

In [4]:
song_df_1 = pd.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

In [5]:
song_df_1.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [6]:
metadata_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'

In [7]:
song_df_2 =  pd.read_csv(metadata_file)

In [8]:
song_merged_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")

In [9]:
len(song_merged_df)

2000000

In [10]:
song_merged_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [11]:
song_df = song_merged_df.head(10000)

In [12]:
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [13]:
#merge title and artist into one column called song
pd.options.mode.chained_assignment = None  # default='warn'

In [14]:
song_df['song'] = song_df['title'].map(str) + '-' + song_df['artist_name']

In [15]:
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,The Cove-Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,Entre Dos Aguas-Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,Stronger-Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,Constellations-Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,Learn To Fly-Foo Fighters


In [16]:
#group by song and take count of how many times the song was listened to by users
song_grouped = song_df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()

In [17]:
#take sum of each song's listen count
lc_grouped_sum = song_grouped['listen_count'].sum()

In [18]:
song_grouped['percentage']  = song_grouped['listen_count'].div(lc_grouped_sum)*100

In [19]:
#songs by popularity
song_grouped.sort_values(['listen_count', 'song'], ascending = [0,1])

,song,listen_count,percentage
3660,Sehr kosmisch-Harmonia,45,0.45
4678,Undo-Björk,32,0.32
5105,You're The One-Dwight Yoakam,32,0.32
1071,Dog Days Are Over (Radio Edit)-Florence + The ...,28,0.28
3655,Secrets-OneRepublic,28,0.28
4378,The Scientist-Coldplay,27,0.27
4712,Use Somebody-Kings Of Leon,27,0.27
3476,Revelry-Kings Of Leon,26,0.26
1387,Fireflies-Charttraxx Karaoke,24,0.24
1862,Horn Concerto No. 4 in E flat K495: II. Romanc...,23,0.23


In [20]:
users = song_df['user_id'].unique()
songs = song_df['song_id'].unique()


In [21]:
len(users)

365

In [22]:
len(songs)

5175

In [23]:
import numpy as np

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
#split data into test and train
train_data, test_data = train_test_split(song_df, test_size=0.20, random_state=0)

In [26]:
print(train_data.head(5))

                                       user_id             song_id  \
7389  94d5bdc37683950e90c56c9b32721edb5d347600  SOXNZOW12AB017F756   
9275  1012ecfd277b96487ed8357d02fa8326b13696a5  SOXHYVQ12AB0187949   
2995  15415fa2745b344bce958967c346f2a89f792f63  SOOSZAZ12A6D4FADF8   
5316  ffadf9297a99945c0513cd87939d91d8b602936b  SOWDJEJ12A8C1339FE   
356   5a905f000fc1ff3df7ca807d57edb608863db05d  SOAMPRJ12A8AE45F38   

      listen_count                 title  \
7389             2      Half Of My Heart   
9275             1  The Beautiful People   
2995             1     Sanctify Yourself   
5316             4     Heart Cooks Brain   
356             20                 Rorol   

                                                release      artist_name  \
7389                                     Battle Studies       John Mayer   
9275             Antichrist Superstar (Ecopac Explicit)   Marilyn Manson   
2995                             Glittering Prize 81/92     Simple Minds   
5316  Ever

In [31]:
#make a popularity based recommender. Recommend songs for the user id given as input
import Recommenders

In [33]:
pr = Recommenders.popularity_recommender_py()
pr.create(train_data, 'user_id', 'song')

In [34]:
#all most listened to songs (according to listen_count sum for each song) are recommended to the user in popularity model
#not personalized. All are recommended the same

In [35]:
user_id = users[5]
pr.recommend(user_id)

,user_id,song,score,Rank
3194,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Sehr kosmisch-Harmonia,37,1.0
4083,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Undo-Björk,27,2.0
931,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Dog Days Are Over (Radio Edit)-Florence + The ...,24,3.0
4443,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You're The One-Dwight Yoakam,24,4.0
3034,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Revelry-Kings Of Leon,21,5.0
3189,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Secrets-OneRepublic,21,6.0
4112,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Use Somebody-Kings Of Leon,21,7.0
1207,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Fireflies-Charttraxx Karaoke,20,8.0
1577,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hey_ Soul Sister-Train,19,9.0
1626,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Horn Concerto No. 4 in E flat K495: II. Romanc...,19,10.0


In [36]:
user_id = users[8]
pr.recommend(user_id)

,user_id,song,score,Rank
3194,9bb911319fbc04f01755814cb5edb21df3d1a336,Sehr kosmisch-Harmonia,37,1.0
4083,9bb911319fbc04f01755814cb5edb21df3d1a336,Undo-Björk,27,2.0
931,9bb911319fbc04f01755814cb5edb21df3d1a336,Dog Days Are Over (Radio Edit)-Florence + The ...,24,3.0
4443,9bb911319fbc04f01755814cb5edb21df3d1a336,You're The One-Dwight Yoakam,24,4.0
3034,9bb911319fbc04f01755814cb5edb21df3d1a336,Revelry-Kings Of Leon,21,5.0
3189,9bb911319fbc04f01755814cb5edb21df3d1a336,Secrets-OneRepublic,21,6.0
4112,9bb911319fbc04f01755814cb5edb21df3d1a336,Use Somebody-Kings Of Leon,21,7.0
1207,9bb911319fbc04f01755814cb5edb21df3d1a336,Fireflies-Charttraxx Karaoke,20,8.0
1577,9bb911319fbc04f01755814cb5edb21df3d1a336,Hey_ Soul Sister-Train,19,9.0
1626,9bb911319fbc04f01755814cb5edb21df3d1a336,Horn Concerto No. 4 in E flat K495: II. Romanc...,19,10.0


In [37]:
#both users are recommended the same songs

In [38]:
#Personalized recommendation - item based collaborative filtering
#content based is prediction derived from past
#collaborative is prediction derived from other similar users - user-item or item-item
iscf = Recommenders.item_similarity_recommender_py()
iscf.create(train_data, 'user_id', 'song')

In [42]:
user_id = users[8]
user_items = iscf.get_user_items(user_id)
print("------------------------------------------------------------------------------------")
print("Training data songs (that this user listens to) for the user userid: %s:" % user_id)
print("------------------------------------------------------------------------------------")

for user_item in user_items:
    print(user_item)

print("----------------------------------------------------------------------")
print("Recommendation process going on:")
print("----------------------------------------------------------------------")

iscf.recommend(user_id)

------------------------------------------------------------------------------------
Training data songs (that this user listens to) for the user userid: 9bb911319fbc04f01755814cb5edb21df3d1a336:
------------------------------------------------------------------------------------
Tell Me Why-Supermode
Raining Again (Steve Angello's Vocal Mix)-Moby
Angel On My Shoulder (EDX Radio Edit)-Kaskade
If I Can't Have You-Mount Sims
----------------------------------------------------------------------
Recommendation process going on:
----------------------------------------------------------------------
No. of unique songs for the user: 4
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :16


,user_id,song,score,rank
0,9bb911319fbc04f01755814cb5edb21df3d1a336,Cover My Eyes-La Roux,0.0,1.0
1,9bb911319fbc04f01755814cb5edb21df3d1a336,The Carpal Tunnel Of Love-Fall Out Boy,0.0,2.0
2,9bb911319fbc04f01755814cb5edb21df3d1a336,The Whole World-Outkast Featuring Killer Mike,0.0,3.0
3,9bb911319fbc04f01755814cb5edb21df3d1a336,Go With The Flow-Queens Of The Stone Age,0.0,4.0
4,9bb911319fbc04f01755814cb5edb21df3d1a336,She Just Likes To Fight-Four Tet,0.0,5.0
5,9bb911319fbc04f01755814cb5edb21df3d1a336,Mourning Air-Portishead,0.0,6.0
6,9bb911319fbc04f01755814cb5edb21df3d1a336,Break Through-Colbie Caillat,0.0,7.0
7,9bb911319fbc04f01755814cb5edb21df3d1a336,Creepin Up The Backstairs-The Fratellis,0.0,8.0
8,9bb911319fbc04f01755814cb5edb21df3d1a336,Warning Sign-Coldplay,0.0,9.0
9,9bb911319fbc04f01755814cb5edb21df3d1a336,Your Arms Feel Like home-3 Doors Down,0.0,10.0


In [41]:
iscf.get_similar_items(['Warning Sign-Coldplay'])

no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :324


,user_id,song,score,rank
0,,She Just Likes To Fight-Four Tet,1.0,1.0
1,,We Never Change-Coldplay,1.0,2.0
2,,Puppetmad-Puppetmastaz,1.0,3.0
3,,God Put A Smile Upon Your Face-Coldplay,1.0,4.0
4,,Susie Q-Creedence Clearwater Revival,1.0,5.0
5,,The Joker-Fatboy Slim,1.0,6.0
6,,Korg Rhythm Afro-Holy Fuck,1.0,7.0
7,,This Unfolds-Four Tet,1.0,8.0
8,,high fives-Four Tet,1.0,9.0
9,,Camel-Flying Lotus,1.0,10.0


In [43]:
#nearest neighbours of the song given as output